In [3]:
from lxml import etree, objectify
import os
import glob
from datetime import datetime
import dateutil.parser
import pytz
import time
import pprint
from business_calendar import Calendar
from ldap3 import Server, Connection, SUBTREE
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
from pyecharts import Bar, Pie, Bar3D
#from ipywidgets import interact
#import ipywidgets as widgets
import plotly.offline as py
py.offline.init_notebook_mode(connected=True)
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.graph_objs import Histogram, Bar, Scatter, Box, Layout, Figure
import math

## 1. omniplan数据解析及处理

In [4]:
class ldapc:
    def __init__(self, ldap_url, ldap_user, ldap_password):
        self.c = self.__init_connection(ldap_url, ldap_user, ldap_password)
    
    def __init_connection(self, ldap_url, ldap_user, ldap_password):
        try:
            server = Server(ldap_url)
            conn = Connection(server, user=ldap_user, password=ldap_password)
            conn.open()
            conn.bind()
        except Exception as e:
            raise e
        return conn

    def search(self, base_dn, condition, attr):
        res = self.c.search(
            search_base = base_dn,
            search_filter = condition, 
            search_scope = SUBTREE, 
            attributes = attr,
            paged_size = 5
        )
        if res:
            entry = self.c.response[0]
            return entry
    
    def find_all_map(self, base_dn, attr):
        res = self.c.search(
            search_base = base_dn,
            search_filter = '(displayName=*)', 
            search_scope = SUBTREE, 
            attributes = attr,
            paged_size = 10000
        )
        if res:
            users = self.c.response
            user_map = {}
            for user in users:
                if 'attributes' in user: 
                    user = user['attributes']
                    user_map[user['displayName']] = user
            return user_map
        
ldap_url='172.20.27.199'
ldap_authuser='CN=dev,CN=Users,DC=ldap,DC=cai-inc,DC=com'
ldap_authpass='ReadOnly@ZcyDEVUser'
base_dn='DC=ldap,DC=cai-inc,DC=com'
ldap_context = ldapc(ldap_url, ldap_authuser, ldap_authpass)
# attributes = ['department', 'displayName', 'mail', 'sAMAccountName', 'employeeID']
attributes = ['department', 'displayName']
user_map = ldap_context.find_all_map(base_dn, attributes)


In [5]:
cur_path = os.path.abspath('.') + '/'
print (cur_path)

/Users/limuqing/Desktop/omniplan-analysis-python/


In [6]:
def cal_effort(tasks):
    '计算每个员工所花工时'
    
    total_effort = 0
    cur_start = 0
    cur_end = 0
    cur_task = {}
    for index in range(len(tasks)):
        task = tasks[index]
        start = task['start_timestamp']
        end = task['end_timestamp']
        effort = task['effort']
        if index == 0:
            cur_start = start
            cur_end = end
            total_effort += effort
        else:
            if start >= cur_end:
                total_effort += effort
                cur_start = start
                cur_end = end
            elif end > cur_end:
                cal = Calendar()
                total_effort += abs(cal.workdaycount(task['end_local_time'], cur_task['end_local_time']) * 8 * 3600)
                cur_end = end
                cur_start = start
            else: 
                cur_start = start
        cur_task = task
                
    return int(total_effort / 3600 / 8)

In [7]:
def parseFile(oplxFileName):
    '解析xml文件，转化为保存每个员工ID、姓名、部门以及工时的dateframe'
    
    metadata = cur_path + oplxFileName + '/Actual.xml'
    parser = etree.XMLParser(remove_blank_text=True)
    tree = etree.parse(metadata, parser) #lxml.etree._ElementTree对象
    root = tree.getroot()#lxml.etree._Element对象
    #去除tag前面的命名空间
    for elem in root.iter():
        i = elem.tag.find('}')
        if i >= 0:
            elem.tag = elem.tag[i+1:]

            
    resources = []
    for staff in root.xpath("//resource/type[text()='Staff']/.."):
        staff_id = staff.get('id')
        staff_name = staff.find('name').text
        try:
            ldap_user = user_map[staff_name] #ldap-user为一个字典，如{'displayName': '一休', 'department': '政采云有限公司,运营部,交易运营,浙江区域及垂管运营'}
            staff = {
                'id': staff_id,
                'name': staff_name
            }
            if 'department' in ldap_user:
                if not isinstance(ldap_user['department'], list):
                    depts_list = ldap_user['department'].split(',')
                    ldap_user['department'] = depts_list[len(depts_list) - 1]
                    staff = dict(**staff, **ldap_user)
                    resources.append(staff)
            else:
                print('Error: {}员工缺少部门信息'.format(ldap_user['displayName']))
        
        except KeyError:
            print('Error: 员工不存在, 文件: {}, 员工名称:{}'.format(oplxFileName, staff_name))

            
    for staff in resources:
        tasks = root.xpath("//task/assignment[@idref='" + staff['id'] + "']/..")
        staff_tasks = []
        for task in tasks:
            try:
                assignment_count = len(task.findall('assignment'))
                effort = int(int(task.find('effort').text) / assignment_count)
                start_local_time = dateutil.parser.parse(task.find('start-constraint-date').text).astimezone(pytz.timezone('Asia/Shanghai')).date()
                cal = Calendar()
                end_local_time = cal.addworkdays(start_local_time, int(effort / 3600 / 8))
                start_timestamp = int(time.mktime(start_local_time.timetuple()))
                end_timestamp = int(time.mktime(end_local_time.timetuple()))
                staff_tasks.append({
                    'start_timestamp': start_timestamp,
                    'start_local_time': start_local_time,
                    'end_timestamp': end_timestamp,
                    'end_local_time': end_local_time,
                    'title': task.find('title').text,
                    'effort': effort
                })
            except AttributeError as e:
                print('Error: 属性缺失, 文件: {}, 任务名称: {}'.format(oplxFileName, task.find('title').text))
        staff_tasks = sorted(staff_tasks, key=lambda k: k['end_timestamp'], reverse=True)
        staff['tasks'] = sorted(staff_tasks, key=lambda k: k['start_timestamp'])
        staff['effort'] = cal_effort(staff['tasks'])
        
    df = pd.DataFrame(resources)
    df = df.dropna(subset=['department'])
    return df

In [8]:
dfs = []
for oplxFile in glob.glob('*.oplx'):
    dfs.append(parseFile(oplxFile))
d = pd.concat(dfs).drop(columns=['id'])

Error: 员工不存在, 文件: 青松-201811.oplx, 员工名称:资源 1
Error: 员工不存在, 文件: 青松-201811.oplx, 员工名称:陈龙
Error: 员工不存在, 文件: 青松-201811.oplx, 员工名称:韦凯驿
Error: 属性缺失, 文件: 青松-201811.oplx, 任务名称: 协议供货_20181106
Error: 属性缺失, 文件: 青松-201811.oplx, 任务名称: 在线询价_20181115
Error: 属性缺失, 文件: 青松-201811.oplx, 任务名称: 在线询价_20181115
Error: 属性缺失, 文件: 青松-201811.oplx, 任务名称: 在线询价_20181115
Error: 员工不存在, 文件: 黄裳-201811.oplx, 员工名称:资源 1
Error: 员工不存在, 文件: 黄裳-201811.oplx, 员工名称:鹰涯
Error: 员工不存在, 文件: 黄裳-201811.oplx, 员工名称:潇湘
Error: 员工不存在, 文件: 黄裳-201811.oplx, 员工名称:扬扬
Error: 属性缺失, 文件: 黄裳-201811.oplx, 任务名称: 采购目录_20181120（采购目录老接口适配）- 开发
Error: 属性缺失, 文件: 黄裳-201811.oplx, 任务名称: 收费策略配置
Error: 员工不存在, 文件: 元芳-201811.oplx, 员工名称:资源 1
Error: 员工不存在, 文件: 元芳-201811.oplx, 员工名称:宪标
Error: 员工不存在, 文件: 元芳-201811.oplx, 员工名称:竹溪
Error: 员工不存在, 文件: 元芳-201811.oplx, 员工名称:元帅
Error: 属性缺失, 文件: 元芳-201811.oplx, 任务名称: 类目层级扩增需求后端
Error: 属性缺失, 文件: 元芳-201811.oplx, 任务名称: 网超对接协议中心后端
Error: 属性缺失, 文件: 元芳-201811.oplx, 任务名称: 调用SPU接口方式改造后端
Error: 属性缺失, 文件: 元芳-201811.oplx, 任务名称: 【前台类目】标签化后端
E

## 2. 数据概览

数据保存为dataframe格式，其结构如下表所示：

In [9]:
table = ff.create_table(d)
py.iplot(table)

In [10]:
d.shape

(56, 5)

共有56位员工信息，每位员工保存有姓名、部门、工时等6个字段信息

In [11]:
d['department'].unique().tolist()

['电子卖场1',
 '平台运营',
 '计划管理',
 '交易载体',
 '支付结算',
 '共享前端',
 '财政管理',
 '共享业务质量保障',
 '政采云驻场',
 '商品中心']

56位员工来自以上10个部门

In [12]:
d['effort'].mean()

18.178571428571427

56位员工的平均工时为18天/月

In [13]:
d['displayName'][d['effort'] == 0].tolist()

['虫二', '泽溥', '灰谷', '渊虹', '凉一', '奕铭', '子木', '一弦']

其中有以上8位员工的工时为0

In [14]:
d['effort'][d['effort'] != 0].mean()

21.208333333333332

剔除工时为0的员工后，剩下员工的平均工时为21天/月

## 3. 数据分析与可视化

### 3.1 员工工时分析

In [15]:
data = [Histogram(
    x = d['effort'],
    #histnorm='probability',
    marker = dict(
        color = '#EB89B5'
    ),
    opacity = 0.7
)]

layout = Layout(
    title = '员工工时分布直方图',
    xaxis=dict(
        title='工时'
    ),
    yaxis=dict(
        title='计数'
    ),
    bargap=0.1,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

从以上员工工时分布直方图中可以看出，工时在10-14天之间的员工最多，有13人；工时超过30天的员工有11人。

In [16]:
def bar_charts(dataframe, x_column, y_column, chart_title = None, x_title = None, y_title = None):
    
    x = dataframe[x_column]
    y = dataframe[y_column]
    
    data = [Bar(
        x = x,
        y = y,
        marker = dict(
        color = '#EB89B5'
        ),
        opacity = 0.7
    )]
    
    layout = Layout(
        title = chart_title,
        xaxis = dict(title = x_title, tickmode = 'linear'),
        yaxis = dict(title = y_title)
    )
    
    fig = Figure(data = data, layout = layout)
    py.iplot(fig)

In [98]:
d1 = d.sort_values(by = ['effort'], ascending=False)
bar_charts(d1, 'displayName', 'effort', '员工工时分布', '员工', '工时' )

以上柱状图为每个员工的工时分布，宇尘，白起，致虑三位员工的所花工时最多，为37天。

In [17]:
#bubble_size = []
#for _, row in d.iterrows():
#    bubble_size.append(math.sqrt(row['effort'])*150)    
#d['size'] = bubble_size
data = []
d3 = d.groupby('department').groups.keys()
for department in d3:
    trace = Bar(
        x = d['displayName'][d['department'] == department],
        y = d['effort'][d['department'] == department],
        name = department,
        opacity = 0.8,
        #mode = 'markers',
        #marker=dict(
            #symbol = 'circle',
            #size = d['size'][d['department'] == department],
            #sizemode = 'area',
            #line = dict(
                #width=2
            #),
        #)
    )
    data.append(trace)

ave_line = Scatter(
    x = d['displayName'],
    y = np.full(d.shape[0], d['effort'].mean()),
    name = '平均工时',
    mode = 'lines',
    line=dict(color = '#FF3333')
)


data.append(ave_line)
    

layout = Layout(
    title='研发工时分布图',
    xaxis=dict(
        title='员工',
        gridcolor='rgb(255, 255, 255)',
        #range=[2.003297660701705, 5.191505530708712],
        #type='log',
        #zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='工时',
        gridcolor='rgb(255, 255, 255)',
        #range=[36.12621671352166, 91.72921793264332],
        #zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

以上柱状图为员工研发工时分布图，不同颜色代表不同的部门，红色水平线代表所有员工的平均工时。平台运营、政采云驻场、财政管理三个部门均只有一个员工，电子卖场1部门拥有最多员工。支付结算部门大多数员工工时超过平均水平，而共享业务质量保障部门大多数员工工时低于平均水平。

### 3.2 部门人均工时分析

In [18]:
d2 = d.groupby(['department'], as_index=False).agg({'displayName': 'count', 'effort': 'sum'})
d2['dept_ave_effort'] = d2['effort']/d2['displayName']
d2 = d2.sort_values(by = ['dept_ave_effort'], ascending=False)

bar_charts(d2, 'department', 'dept_ave_effort', '部门人均工时分布', '部门', '人均工时' )

以上柱状图为每个部门的人均工时分布，支付结算部门的人均工时最多，为27天，而商品中心部门的人均工时最少，为10.4天

In [19]:
data = []
d3 = d.groupby('department').groups.keys()
for department in d3:
    trace = Box(
        y = d['effort'][d['department'] == department],
        name = department,
    )
    data.append(trace)

medium_line = Scatter(
    x = d['department'].unique(),
    y = np.full(len(d['department'].unique()), d['effort'].quantile(q=0.5)),
    name = '全体员工工时中位数',
    mode = 'lines',
    line=dict(color = '#FF3333')
)

data.append(medium_line)
 
py.iplot(data)

以上图形为各部门员工工时分布的箱线图，其中：   
支付结算、计划管理、交易载体这3个部门员工研发工时较多，高于全体中位数，并且员工工时分布比较集中；  
共享前端部门员工研发工时中位数高于全体中位数，但是员工工时分布比较分散，有部分员工工时较少；  
电子卖场1部门员工研发工时中位数与全体中位数持平；  
共享业务质量保证部门所有员工研发工时低于全体中位数，并且员工工时分布比较集中；  
商品中心部门员工研发工时中位数远低于全体中位数，员工工时分布比较分散。

### 3.3 3D图总体分析

In [26]:
bar3d = Bar3D("研发工时分布图", width=1200, height=600)
x_axis = d['displayName']
y_axis = d['department'].unique()
data = []
for index, row in d.iterrows():
    data.append([row['displayName'], row['department'], row['effort']])
bar3d.add(
    "",
    x_axis,
    y_axis,
    data,
    is_visualmap=True,
    visual_range=[0, 30],
    grid3d_width=200,
    grid3d_depth=80,
    grid3d_shading="realistic"
)
bar3d

以上3D图可直观地展现每个员工的所属部门，以及研发工时是否太多或者太少。

## 4. 结论

1. 剔除工时为0的员工后，全体员工的平均研发工时为21天，处在正常水平；  
2. 支付结算部门员工的研发压力最大，7人中有5人的工时超过20天，其中海根、修远、白起3人的研发工时超过35天，承担着过大的压力，建议可以把这三人的一些任务分摊给同部门研发工时较少的同事鲲鹏和游侠； 
3. 计划管理和交易载体两个部门的研发压力其次，并且同样存在任务分配不均问题； 
4. 电子卖场部门的任务分配不均问题严重，有5人的工时超过30天，其中致虑和宇尘的工时为37天，同时有8人工时少于20天，3人工时为0，建议合理分配任务，降低致虑和宇尘等同学的压力； 
5. 共享前端部门的研发压力中等，部门平均工时在25天左右，但有少部分人的工时过少； 
6. 商品中心部门的研发压力较小，但也存在任务分配不均问题； 
7. 共享业务质量保证部门的研发压力最小，普遍低于全体平均水平； 
